In [1]:
import pandas as pd
import numpy as np
import re
import ast
import json
import os
import pymysql

from pandasql import sqldf
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine, text  




# Set options to display all columns and rows without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)

/Users/danielriggi/Library/Python/3.8/lib/python/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def create_ingredient_list(row):
    return ast.literal_eval(row['Cleaned_Ingredients'])


# Define a function to remove non-letter characters from a string
def apply_regex(input_string):

    #remove anything that isnt a letter
    letter_pattern = r'[^A-Za-z\s-]'
    input_string = re.sub(letter_pattern, '', input_string)

    #replace '-' seperatley
    dash_pattern = r'-'
    input_string = re.sub(dash_pattern, ' ', input_string)

    #remove units of measurement
    unit_pattern = r'\b(?:tablespoon|teaspoon|oz|cup|tsp|tbsp|lb|pound|g|ml|ounce|fl|about|optional|room temperature|divided|pint)s?\b'
    input_string = re.sub(unit_pattern, '', input_string, flags=re.IGNORECASE)
    input_string = re.sub(r'\s{2, }', '', input_string)
    return input_string.strip()



def get_similarity_score_token_sort(ingredient1, ingredient2):
    return fuzz.token_sort_ratio(ingredient1.lower(), ingredient2.lower())




recipe_df = pd.read_csv('../data/recipe_data.csv')
recipe_df = recipe_df.drop(columns=['Unnamed: 0'])

recipe_df['Cleaned_Ingredients'] = recipe_df.apply(lambda row: create_ingredient_list(row), axis = 1)
recipe_df['id'] = recipe_df.index + 5
recipe_df = recipe_df.loc[~recipe_df['Instructions'].isna(), :]
recipe_df_original = recipe_df.copy()

with open('../data/train.json', 'r') as file:
    train = json.load(file)

with open('../data/test.json', 'r') as file:
    test = json.load(file)


unique_ingredients = set()
ingredient_data = []

for recipe in test:
    for ingredient in recipe['ingredients']:
        unique_ingredients.add(ingredient)

for recipe in train:
    for ingredient in recipe['ingredients']:
        unique_ingredients.add(ingredient)

data = [{'id': i, 'name': name.lower()} for i, name in enumerate(unique_ingredients, start=5)]
ingredient_df = pd.DataFrame(data)


# Use the explode function to expand the list of ingredients
recipe_df = recipe_df[['Title', 'id', 'Cleaned_Ingredients']].explode('Cleaned_Ingredients', ignore_index=True)
recipe_df['Cleaned_Ingredients'] = recipe_df['Cleaned_Ingredients'].str.lower()

# Rename the column if needed
recipe_df = recipe_df.rename(columns={'Cleaned_Ingredients': 'ingredient'})

# Apply the function to the DataFrame column
recipe_df['regex_ingredient'] = recipe_df['ingredient'].apply(apply_regex)


best_matches_ids = []
best_matches_names = []
best_matches_scores = []

for recipe_ingredient in recipe_df['regex_ingredient']:
    ingredient_df_sample = ingredient_df[ingredient_df['name'].apply(lambda x: x in recipe_ingredient or x.rstrip('s') in recipe_ingredient)]

    if ingredient_df_sample.shape[0] > 0:
        best_match = max(ingredient_df_sample['name'], key=lambda ingredient: get_similarity_score_token_sort(recipe_ingredient, ingredient))
        # Calculate and store the similarity score
        similarity_score = get_similarity_score_token_sort(recipe_ingredient, best_match)

        #remove recipe/ingredient combinations with a low similarity score
        if similarity_score > 15.0:
            best_match_id = ingredient_df_sample.loc[ingredient_df_sample['name'] == best_match, 'id'].values[0]
            best_matches_ids.append(best_match_id)
            best_matches_names.append(best_match)
        else:
            best_matches_ids.append(None)
            best_matches_names.append(None)
    else:
        best_matches_ids.append(None)
        best_matches_names.append(None)

# Add the best matches and scores to the recipe_df DataFrame
recipe_df['best_match_id'] = best_matches_ids
recipe_df['best_match_name'] = best_matches_names

recipe_df = recipe_df.loc[~recipe_df['best_match_id'].isna(), :]
recipe_df = recipe_df.rename(columns={'id': 'recipe_id', 'best_match_id': 'ingredient_id'})
recipe_df['ingredient_id'] = recipe_df['ingredient_id'].astype(int)

ingredient_df = ingredient_df.loc[ingredient_df['id'].isin(recipe_df['ingredient_id'])]
recipe_df_original = recipe_df_original.loc[recipe_df_original['id'].isin(recipe_df['recipe_id'])]

recipe_df = recipe_df.loc[:, ['recipe_id', 'ingredient_id']]

# Replace these with your MySQL database credentials
db_username = 'user'
db_password = 'user'
db_host = 'localhost'
db_port = '3306'
db_name = 'reciperecommenderdb'

connection = pymysql.connect(host=db_host, user=db_username, password=db_password, database=db_name)
cursor = connection.cursor()

recipe_table_name = "recipe"
ingredient_table_name = "ingredient"
recipe_ingredient_table_name = "recipe_ingredient"

r_insert_query = f"INSERT INTO {recipe_table_name} (id, name, description, ingredient_description, recipe_image) VALUES (%s, %s, %s, %s, %s)"
i_insert_query = f"INSERT INTO {ingredient_table_name} (id, name) VALUES (%s, %s)"
ri_insert_query = f"INSERT INTO {recipe_ingredient_table_name} (recipe_id, ingredient_id) VALUES (%s, %s)"

try:
    #recipe table insert
    recipe_data = []
    for idx, row in recipe_df_original.iterrows():
        recipe_data.append(tuple(row[['id', 'Title', 'Instructions', 'Ingredients', 'Image_Name']]))
    cursor.executemany(r_insert_query, recipe_data)
    
    ingredient_data = []
    for idx, row in ingredient_df.iterrows():
        ingredient_data.append(tuple(row[['id', 'name']]))
    cursor.executemany(i_insert_query, ingredient_data)
    
    recipe_ingredient_data = []
    for idx, row in recipe_df.iterrows():
        recipe_ingredient_data.append(tuple(row[['recipe_id', 'ingredient_id']]))
    recipe_ingredient_data = list(set(recipe_ingredient_data))
    cursor.executemany(ri_insert_query, recipe_ingredient_data)
    connection.commit()
    print(f'DataFrame inserted into {recipe_table_name} table in the {db_name} database.')
    print(f'DataFrame inserted into {ingredient_table_name} table in the {db_name} database.')
    print(f'DataFrame inserted into {recipe_ingredient_table_name} table in the {db_name} database.')
except Exception as e:
    # Handle the exception (e.g., log the error)
    print(f"Error: {e}")
finally:
    # Close cursor and connection in the finally block to ensure they are always closed
    cursor.close()
    connection.close()



DataFrame inserted into recipe table in the reciperecommenderdb database.
DataFrame inserted into ingredient table in the reciperecommenderdb database.
DataFrame inserted into recipe_ingredient table in the reciperecommenderdb database.
